<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [12]:
##### Your Code Here #####
import pandas as pd
pd.set_option('display.max_columns', 500)

In [102]:
telecom_data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv', na_values=' ')
telecom_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [103]:
telecom_data.MultipleLines.value_counts()

No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64

In [104]:
telecom_data.InternetService.value_counts()

Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64

In [105]:
telecom_data['TotalCharges'].dropna()
telecom_data['TotalCharges'].value_counts()

20.20      11
19.75       9
19.65       8
20.05       8
19.90       8
           ..
1066.15     1
249.95      1
8333.95     1
7171.70     1
1024.00     1
Name: TotalCharges, Length: 6530, dtype: int64

In [106]:
telecom_data['TotalCharges'] = telecom_data['TotalCharges'].astype('float64')

In [107]:
X = telecom_data.drop(columns='Churn')
y = telecom_data['Churn']

In [108]:
import category_encoders as ce
encoder = ce.OrdinalEncoder()

In [109]:
encode_X = encoder.fit_transform(X)
encode_X.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,29.85,29.85
1,2,2,0,2,1,34,2,2,1,2,2,2,1,1,1,2,2,2,56.95,1889.50
2,3,2,0,2,1,2,2,2,1,2,1,1,1,1,1,1,1,2,53.85,108.15
3,4,2,0,2,1,45,1,1,1,2,2,2,2,1,1,2,2,3,42.30,1840.75
4,5,1,0,2,1,2,2,2,2,1,2,1,1,1,1,1,1,1,70.70,151.65


In [110]:
encode_X.drop(columns='customerID')

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,29.85,29.85
1,2,0,2,1,34,2,2,1,2,2,2,1,1,1,2,2,2,56.95,1889.50
2,2,0,2,1,2,2,2,1,2,1,1,1,1,1,1,1,2,53.85,108.15
3,2,0,2,1,45,1,1,1,2,2,2,2,1,1,2,2,3,42.30,1840.75
4,1,0,2,1,2,2,2,2,1,2,1,1,1,1,1,1,1,70.70,151.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2,0,1,2,24,2,3,1,2,2,2,2,2,2,2,1,2,84.80,1990.50
7039,1,0,1,2,72,2,3,2,1,1,2,1,2,2,2,1,4,103.20,7362.90
7040,1,0,1,2,11,1,1,1,2,2,1,1,1,1,1,1,1,29.60,346.45
7041,2,1,1,1,4,2,3,2,1,2,1,1,1,1,1,1,2,74.40,306.60


In [111]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(encode_X, y, test_size=0.33, random_state=66)

In [112]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

In [113]:
y_train = y_train.replace({'No': 0, 'Yes': 1})
y_test = y_test.replace({'No': 0, 'Yes': 1})

In [114]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [115]:
def create_model(learning_rate, neurons, momentum):
    model = Sequential()
    model.add(Dense(neurons, input_dim=20, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = SGD(learning_rate=learning_rate, momentum=momentum)
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [10],
             'epochs': [10],
             'learning_rate': [.1],
              'neurons': [15],
              'momentum': [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
             }

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.7318779211167614 using {'batch_size': 10, 'epochs': 10, 'learning_rate': 0.1, 'momentum': 0.0, 'neurons': 15}
Means: 0.7318779211167614, Stdev: 0.0014601158393603483 with: {'batch_size': 10, 'epochs': 10, 'learning_rate': 0.1, 'momentum': 0.0, 'neurons': 15}
Means: 0.7318779211167614, Stdev: 0.0014601158393603483 with: {'batch_size': 10, 'epochs': 10, 'learning_rate': 0.1, 'momentum': 0.2, 'neurons': 15}
Means: 0.7318779211167614, Stdev: 0.0014601158393603483 with: {'batch_size': 10, 'epochs': 10, 'learning_rate': 0.1, 'momentum': 0.4, 'neurons': 15}
Means: 0.7318779211167614, Stdev: 0.0014601158393603483 with: {'batch_size': 10, 'epochs': 10, 'learning_rate': 0.1, 'momentum': 0.6, 'neurons': 15}
Means: 0.7318779211167614, Stdev: 0.0014601158393603483 with: {'batch_size': 10, 'epochs': 10, 'learning_rate': 0.1, 'momentum': 0.8, 'neurons': 15}
Means: 0.7318779211167614, Stdev: 0.0014601158393603483 with: {'batch_size': 10, 'epochs': 10, 'learning_rate': 0.1, 'momentum': 0.9, 'ne

In [ ]:
# Set params
epochs = 10
batch_size = 10
learning_rate = 0.1

# Build a model to change activation functions,
# optimizers, metrics, neurons, etc.
model2 = Sequential()
model2.add(Dense(neurons, input_dim=20, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

# Bring out the adam
adam = Adam(learning_rate=learning_rate, )

# Compile the function
model2.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
model2.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=epochs,
          batch_size=batch_size,
          learning_rate=learning_rate,
          )

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?